# Hipotetical Personal Computer (HPC)

O HPC01 foi construido utilizando a arquitetura vista em sala de aula:

![](imagens/s_arq.svg)


O diagrama abaixo descreve os estados de um processador durante a execução de instruções.

![](imagens/fluxo_c.svg)

1. Busca da Instrução (Instruction Fetch): Nesse estágio, o processador busca a próxima instrução a ser executada na memória principal. Ele obtém o endereço da instrução e a carrega para o registrador de instrução.
1. Decodificação do Operando (Operand Decode): Após obter a instrução, o processador a decodifica para entender qual operação deve ser realizada. Ele identifica os registradores envolvidos, o tipo de operação (aritmética, lógica, etc.) e os modos de endereçamento.
1. Busca do Operando (Operand Fetch): Se a instrução requer dados de memória ou registradores, o processador busca esses operandos. Por exemplo, se a instrução for uma adição, ele buscará os valores dos registradores envolvidos.
1. Cálculo do Endereço do Operando (Calculation of Operand Address): Se o operando estiver em memória, o processador calcula o endereço físico correspondente com base no modo de endereçamento. Isso pode envolver somar um deslocamento a um registrador-base ou aplicar outras transformações.
1. Execução: Nesse estágio, a operação é efetivamente realizada. Por exemplo, se a instrução for uma adição, os valores dos operandos são somados.
1. Armazenamento do Operando (Operand Storage): Se a instrução resultar em um valor que precisa ser armazenado (por exemplo, o resultado de uma operação), o processador atualiza os registradores ou a memória conforme necessário.

O ciclo continua, com o processador buscando a próxima instrução após a conclusão da atual. 




## Instruções

As instruções que o HPC02 executará residem na memória RAM do PC e em razão desse fato vamos criar o `fetch instructions` que recupera a isntrução da memória e incia o fluxo de processamento.

In [ ]:
## recupera a informação da memória considerando o PC
def fetch(pc,memory):                               
    ir = memory[pc]                                 # le a instrucao e armazena do (ir)
    pc = pc + 1                                     # incrementa o PC
    return(ir >> 8, ir & 0xFF)                      # o que isso faz?

O HPC01 suporta instruções de 12bits

![](imagens/instrucao1_detalhe.svg)


## Decodificação e Execução das Instruções

Nessa etapa vamos construir o HPC01 para dar suporte as 3 operações vistas em sala de aula, descritas na tabela abaixo.

| OpCode | Instrução  | Parametro | Descrição |
| :-: | :-:       | :-         | :-        |
| 1111 | SHUTDOWN  | -        | Encerra a execução |
| 0001 | LOAD      | inteiro 8bits        | Carrega o valor no registrador r0 |
| 0010 | ADD      | inteiro 8bits        | Soma o valor do endereço de memória a r0 |
| 0011 | SUB      | inteiro 8bits        | Subtrai o valor do endereço de memória a r0 |
| 0100 | BEQ      | inteiro 8bits        | Branch para o operando |

---
**BEQ** : Se o resultado da ultima subtração for zero va para o endereço determinado no Operando

![](imagens/ULAF-FULL.png)

```python
 if z == True:
    pc = operando
```

---

primeiro PC e colocar pc na posição correta

In [ ]:
mem = [0] * 16                                                      # Setup 16 enderecoes de memoria
#INST 01 LOAD 
mem[0] = 0b000100001001
#SUB 9
mem[1] = 0b001100000111                     
# BEQ
mem[2] = 0b010000000110
#SHUTDOWN
mem[3] = 0b111100000000
mem[6] = 0b111100000000
#DADOS
mem[7] = 9


In [ ]:
mem

construindo o PC

In [ ]:
pc = 0                                                              
run = 1
z=0

while run == 1:                             
    # fecth da memoria
    opCode, address = fetch(pc,mem)                                
    pc_anterior = pc
    pc += 1
    # shutdown
    if opCode == 0b1111: 
        run = 0
    # load
    elif opCode == 0b0001:                                          
        r0 = address                                                
    # add
    elif opCode == 0b0010:
        mar = address                                               
        mbr = mem[mar]                                              
        r0 = mbr + r0  
    # subtract
    elif opCode == 0b0011:
        r0 = r0 - mem[address]
        # flag z
        if r0 == 0: 
            z = 1  
        else: 
            z = 0
    # beq (branch)
    elif opCode == 0b0100:
        # If BEQ load PC on zero flag
        if z == 1:
            pc = address                                 
    
    
    print('pc = ',pc_anterior,'opCode =',opCode,'\tRegister r0 =',r0,'z = ',z)